Attempt at trying and learning the Flood Dataset in Kaggle [[Here](https://www.kaggle.com/competitions/playground-series-s4e5/code)]

In [5]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

import pandas as pd
import seaborn as sns 
import numpy as np
train = pd.read_csv('datasets/floodtrain.csv')

test = pd.read_csv('datasets/floodtest.csv')

train

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,0,5,8,5,8,6,4,4,3,3,...,5,3,3,5,4,7,5,7,3,0.445
1,1,6,7,4,4,8,8,3,5,4,...,7,2,0,3,5,3,3,4,3,0.450
2,2,6,5,6,7,3,7,1,5,4,...,7,3,7,5,6,8,2,3,3,0.530
3,3,3,4,6,5,4,8,4,7,6,...,2,4,7,4,4,6,5,7,5,0.535
4,4,5,3,2,6,4,4,3,3,3,...,2,2,6,6,4,1,2,3,5,0.415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117952,1117952,3,3,4,10,4,5,5,7,10,...,7,8,7,2,2,1,4,6,4,0.495
1117953,1117953,2,2,4,3,9,5,8,1,3,...,9,4,4,3,7,4,9,4,5,0.480
1117954,1117954,7,3,9,4,6,5,9,1,3,...,5,5,5,5,6,5,5,2,4,0.485
1117955,1117955,7,3,3,7,5,2,3,4,6,...,6,8,5,3,4,6,7,6,4,0.495


Let's create a test set

In [ ]:
from sklearn.model_selection import train_test_split


target = train['FloodProbability']
train = train.drop('FloodProbability', axis=1)

X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=.3, random_state=42)


from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, SGDRegressor

pipeline = make_pipeline(StandardScaler(), 
                         PolynomialFeatures(degree=3),
                         SGDRegressor())

num = 10000
pipeline.fit(X_train[:num], y_train[:num])

params = {
    "polynomialfeatures__degree": list(range(1,2)),
    "linearregression__fit_intercept": [False, True],
}

sgd_params = {
    "polynomialfeatures__degree": list(range(1,2)),
    "sgdregressor__max_iter": np.linspace(200,800,50)
    
}

gs = GridSearchCV(pipeline, sgd_params, refit=True)

num = 10000
gs.fit(X_train[:num], y_train[:num])

from sklearn.metrics import r2_score

y_pred = gs.predict(X_test)

print(f'The R2 score is {r2_score(y_pred, y_test)}')


In [ ]:

gs.best_params_

In [ ]:

test_results = gs.predict(test)
submission = pd.DataFrame([ test['id'], test_results], columns=['id', 'FloodProbability'])
submission['id'] = test['id']

submission.to_csv("floods.csv", header=True, index=False)
